# <a id="0">Wine Data Exercises (Part 2) with pipeline/column transformer and Cross Validation</a>

In this notebook, we will review basic steps of exploratory data analysis following the example in the EDA-PIPELINE,ipynb example. We will work with the wine data set __winequality-white.csv__ provided in the data folder. 

__Dataset schema:__ 
   - fixed acidity
   - volatile acidity
   - citric acid
   - residual sugar
   - chlorides
   - free sulfur dioxide
   - total sulfur dioxide
   - density
   - pH
   - sulphates
   - alcohol

   Output variable (based on sensory data): 
   - quality (score between 0 and 10)

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
  
df = pd.read_csv('../data/winequality-white.csv', sep=';')

In [2]:
fixed_acidity_bin_labels_5 = ['poor', 'average', 'ok', 'good', 'best']
df['fixed acidity group'] = pd.qcut(df['fixed acidity'], q=[0, .2, .4, .6, .8, 1], labels=fixed_acidity_bin_labels_5)
df['fixed acidity group'].value_counts()

fixed acidity group
poor       1107
ok         1017
average     984
good        903
best        887
Name: count, dtype: int64

In [3]:
print(df.columns)


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'fixed acidity group'],
      dtype='object')


In [4]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   fixed acidity         4898 non-null   float64 
 1   volatile acidity      4898 non-null   float64 
 2   citric acid           4897 non-null   float64 
 3   residual sugar        4896 non-null   float64 
 4   chlorides             4898 non-null   float64 
 5   free sulfur dioxide   4897 non-null   float64 
 6   total sulfur dioxide  4896 non-null   float64 
 7   density               4898 non-null   float64 
 8   pH                    4897 non-null   float64 
 9   sulphates             4897 non-null   float64 
 10  alcohol               4898 non-null   float64 
 11  quality               4898 non-null   int64   
 12  fixed acidity group   4898 non-null   category
dtypes: category(1), float64(11), int64(1)
memory usage: 464.3 KB


In [5]:
df.shape

(4898, 13)

In [6]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,fixed acidity group
0,7.0,0.27,0.36,NaN,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,ok
1,6.3,0.30,0.34,NaN,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,average
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,best
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,good
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,good


We will look at number of rows, columns and some simple statistics of the dataset using [df.info()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html)

In [7]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             1
residual sugar          2
chlorides               0
free sulfur dioxide     1
total sulfur dioxide    2
density                 0
pH                      1
sulphates               1
alcohol                 0
quality                 0
fixed acidity group     0
dtype: int64

In [8]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4897.000000,4896.000000,4898.000000,4897.000000,4896.000000,4898.000000,4897.000000,4897.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334213,6.389471,0.045772,35.306923,138.363051,0.994027,3.188256,0.489877,10.514267,5.877909
std,0.843868,0.100795,0.121023,5.068507,0.021848,17.008680,42.506522,0.002991,0.151014,0.114117,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


Create a catagorical feature column using qcut()

## Data Processing with Pipeline
 
__Part 1.__ Build a pipeline that has two pre-processors

- One is to impute the missing values with the mean using sklearn's SimpleImputer, scale the numerical features to have similar orders of magnitude by bringing them into the 0-1 range with sklearn's MinMaxScaler, for numerical features 

- One is to use one hot encoding to encode the catagorical feature. Note here even the feature `fixed acidity group` is ordinal, so make sure you choose the most appropriate encoder. 

Then we add a decision tree estimator to form the pipeline. Visualize pipeline. 

In [9]:
df.shape

(4898, 13)

In [10]:
model_features = df.columns.drop(['quality','fixed acidity'])
model_target = 'quality'

print('Model features: ', model_features)
print('Model target: ', model_target)

Model features:  Index(['volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol', 'fixed acidity group'],
      dtype='object')
Model target:  quality


In [11]:
import numpy as np
numerical_features_all = df[model_features].select_dtypes(include=np.number).columns
print('Numerical columns:',numerical_features_all)

print('')

categorical_features_all = df[model_features].select_dtypes(include='category').columns
print('Categorical columns:',categorical_features_all)

Numerical columns: Index(['volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol'],
      dtype='object')

Categorical columns: Index(['fixed acidity group'], dtype='object')


In [12]:
 #Impute numerical columns by using the mean per column to replace the nans

from sklearn.impute import SimpleImputer

 #make another copy of our current dataframe
df_sklearn_imputed = df.copy()
df_sklearn_imputed[numerical_features_all].isnull().sum()

volatile acidity        0
citric acid             1
residual sugar          2
chlorides               0
free sulfur dioxide     1
total sulfur dioxide    2
density                 0
pH                      1
sulphates               1
alcohol                 0
dtype: int64

In [13]:
imputer = SimpleImputer(strategy='mean')
df_sklearn_imputed[numerical_features_all] = imputer.fit_transform(df_sklearn_imputed[numerical_features_all])

print(df_sklearn_imputed[numerical_features_all].isnull().sum())

volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
dtype: int64


In [14]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

In [15]:
print('Training set shape:', train_data.shape)

print('Class 6 samples in the training set:', sum(train_data[model_target] == 6))
print('Class 5 samples in the training set:', sum(train_data[model_target] == 5))
print('Class 7 samples in the training set:', sum(train_data[model_target] == 7))
print('Class 8 samples in the training set:', sum(train_data[model_target] == 8))
print('Class 4 samples in the training set:', sum(train_data[model_target] == 4))
print('Class 3 samples in the training set:', sum(train_data[model_target] == 3))
print('Class 9 samples in the training set:', sum(train_data[model_target] == 9))

Training set shape: (3918, 13)
Class 6 samples in the training set: 1766
Class 5 samples in the training set: 1166
Class 7 samples in the training set: 688
Class 8 samples in the training set: 140
Class 4 samples in the training set: 138
Class 3 samples in the training set: 15
Class 9 samples in the training set: 5


In [16]:
from sklearn.utils import shuffle

unique_classes = train_data[model_target].unique()
upsampled_classes = {}
desired_sample_size = len(train_data[train_data[model_target] == unique_classes[0]])

for class_label in unique_classes:
    class_data = train_data[train_data[model_target] == class_label]
    upsampled_class_data = class_data.sample(n=desired_sample_size, replace=True, random_state=42)
    upsampled_classes[class_label] = upsampled_class_data

train_data_balanced = pd.concat(list(upsampled_classes.values()))
train_data_balanced = shuffle(train_data_balanced)


In [17]:
print('Training set shape:', train_data_balanced.shape)

#Calculate the number of unique classes
n_classes = len(train_data_balanced[model_target].unique())

#Loop through each class label
for class_label in [3, 4, 5, 6, 7, 8, 9]:
    # Count the number of samples in the training set for the current class
    class_samples_train = sum(train_data_balanced[model_target] == class_label)
    # Print the count of samples for the current class in the training set
    print(f'Class {class_label} samples in the training set:', class_samples_train)


Training set shape: (12362, 13)
Class 3 samples in the training set: 1766
Class 4 samples in the training set: 1766
Class 5 samples in the training set: 1766
Class 6 samples in the training set: 1766
Class 7 samples in the training set: 1766
Class 8 samples in the training set: 1766
Class 9 samples in the training set: 1766


In [18]:
numerical_features_all

Index(['volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol'],
      dtype='object')

In [19]:
categorical_features_all = df[model_features].select_dtypes(include='category').columns

In [20]:
categorical_features_all

Index(['fixed acidity group'], dtype='object')

In [21]:
df[categorical_features_all].head(10)

,fixed acidity group
0,ok
1,average
2,best
3,good
4,good
5,best
6,poor
7,ok
8,average
9,best


In [22]:
# can also grab less numerical features, as some numerical data might not be very useful
numerical_features = ['volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']

# dropping the IDs features, RescuerID and PetID here 
categorical_features = ['fixed acidity group']

In [23]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


# Preprocess the numerical features
numerical_processor = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean')),
    ('num_scaler', MinMaxScaler()) # Shown in case is needed, not a must with Decision Trees
])
                  
# Preprocess the categorical features
categorical_processor = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('cat_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

In [24]:
data_preprocessor = ColumnTransformer([
    ('numerical_pre', numerical_processor, numerical_features),
    ('categorical_pre', categorical_processor, categorical_features)
]) 

In [25]:
# Pipeline desired all data transformers, along with an estimator at the end
# Later you can set/reach the parameters using the names issued - for hyperparameter tuning, for example
pipeline = Pipeline([
    ('data_preprocessing', data_preprocessor),
    #('knn', KNeighborsClassifier()) # n_neighbors defaults to 5
    ('dt', DecisionTreeClassifier())
])

# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('numerical_pre',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer()),
                                                                  ('num_scaler',
                                                                   MinMaxScaler())]),
                                                  ['volatile acidity',
                                                   'citric acid',
                                                   'residual sugar',
                                                   'chlorides',
                                                   'free sulfur dioxide',
                                                   'total sulfur dioxide',
                                                   'density', 'pH', 'sulphates',
                                                   'alcohol']),
                                                 ('categorical_pre',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('cat_encoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['fixed acidity group'])])),
                ('dt', DecisionTreeClassifier())])

In [26]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Get train data to train the pipeline
X_train = train_data[model_features]
y_train = train_data[model_target]

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train)


Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('numerical_pre',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer()),
                                                                  ('num_scaler',
                                                                   MinMaxScaler())]),
                                                  ['volatile acidity',
                                                   'citric acid',
                                                   'residual sugar',
                                                   'chlorides',
                                                   'free sulfur dioxide',
                                                   'total sulfur dioxide',
                                                   'density', 'pH', 'sulphates',
                                                   'alcohol']),
                                                 ('categorical_pre',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('cat_encoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['fixed acidity group'])])),
                ('dt', DecisionTreeClassifier())])

In [27]:
# Use the fitted pipeline to make predictions on the train dataset
train_predictions = pipeline.predict(X_train)
print(confusion_matrix(y_train, train_predictions))
print(classification_report(y_train, train_predictions))
print("Accuracy (training):", accuracy_score(y_train, train_predictions))

[[  15    0    0    0    0    0    0]
 [   0  138    0    0    0    0    0]
 [   0    0 1166    0    0    0    0]
 [   0    0    0 1766    0    0    0]
 [   0    0    0    0  688    0    0]
 [   0    0    0    0    0  140    0]
 [   0    0    0    0    0    0    5]]
              precision    recall  f1-score   support

           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00       138
           5       1.00      1.00      1.00      1166
           6       1.00      1.00      1.00      1766
           7       1.00      1.00      1.00       688
           8       1.00      1.00      1.00       140
           9       1.00      1.00      1.00         5

    accuracy                           1.00      3918
   macro avg       1.00      1.00      1.00      3918
weighted avg       1.00      1.00      1.00      3918

Accuracy (training): 1.0


In [28]:
# Get test data to test the pipeline
X_test = test_data[model_features]
y_test = test_data[model_target]

# Use the fitted pipeline to make predictions on the test dataset
test_predictions = pipeline.predict(X_test)
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))
print("Accuracy (test):", accuracy_score(y_test, test_predictions))

[[  0   1   2   2   0   0   0]
 [  0   7   6  10   0   2   0]
 [  0  14 191  75  10   1   0]
 [  0  13  81 270  55  12   1]
 [  0   1   2  61 116  12   0]
 [  0   0   0   6  11  18   0]
 [  0   0   0   0   0   0   0]]
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         5
           4       0.19      0.28      0.23        25
           5       0.68      0.66      0.67       291
           6       0.64      0.62      0.63       432
           7       0.60      0.60      0.60       192
           8       0.40      0.51      0.45        35
           9       0.00      0.00      0.00         0

    accuracy                           0.61       980
   macro avg       0.36      0.38      0.37       980
weighted avg       0.62      0.61      0.62       980

Accuracy (test): 0.6142857142857143


__Part 2.__ Test the pipeline on the training data, then on the test data.

__Part 3.__ Use 5-fold Grid Search to tune the hyper-parameter for the decision tree estimator. You may use a grid like this

        param_grid={'dt__max_depth': [100, 200, 300],#, 50, 75, 100, 125, 150, 200, 250], 
            'dt__min_samples_leaf': [5, 10, 15],#, 25, 30],
            'dt__min_samples_split': [2, 5, 15]#, 25, 30, 45, 50]
        }


In [29]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

### PIPELINE GRID_SEARCH ###
############################

# Parameter grid for GridSearch
param_grid = {'dt__max_depth': [100, 200, 300],#, 50, 75, 100, 125, 150, 200, 250], 
        'dt__min_samples_leaf': [5, 10, 15],#, 25, 30],
        'dt__min_samples_split': [2, 5, 15]#, 25, 30, 45, 50]
    }

grid_search = GridSearchCV(pipeline, # Base model
                           param_grid, # Parameters to try
                           cv = 3, # Apply 3-fold cross validation
                           n_jobs = -1 # Use all available processors
                        )

# Fit the GridSearch to our training data
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('data_preprocessing',
                                        ColumnTransformer(transformers=[('numerical_pre',
                                                                         Pipeline(steps=[('num_imputer',
                                                                                          SimpleImputer()),
                                                                                         ('num_scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['volatile '
                                                                          'acidity',
                                                                          'citric '
                                                                          'acid',
                                                                          'residual '
                                                                          'sugar',
                                                                          'chlorides',
                                                                          'free '
                                                                          'sulfur '
                                                                          'dioxide',
                                                                          'total '
                                                                          'sulfur '
                                                                          'dioxide',
                                                                          'density',
                                                                          'pH',
                                                                          'sulphates',
                                                                          'alcohol']),
                                                                        (...
                                                                         Pipeline(steps=[('cat_imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('cat_encoder',
                                                                                          OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                         unknown_value=-1))]),
                                                                         ['fixed '
                                                                          'acidity '
                                                                          'group'])])),
                                       ('dt', DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'dt__max_depth': [100, 200, 300],
                         'dt__min_samples_leaf': [5, 10, 15],
                         'dt__min_samples_split': [2, 5, 15]})

In [30]:
# Get the best model out of GridSearchCV
classifier = grid_search.best_estimator_

# Fit the best model to the train data once more
classifier.fit(X_train, y_train)
#print(classifier)

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('numerical_pre',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer()),
                                                                  ('num_scaler',
                                                                   MinMaxScaler())]),
                                                  ['volatile acidity',
                                                   'citric acid',
                                                   'residual sugar',
                                                   'chlorides',
                                                   'free sulfur dioxide',
                                                   'total sulfur dioxide',
                                                   'density', 'pH', 'sulphates',
                                                   'alcohol']),
                                                 ('categorical_pre',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('cat_encoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['fixed acidity group'])])),
                ('dt',
                 DecisionTreeClassifier(max_depth=100, min_samples_leaf=5,
                                        min_samples_split=5))])

In [31]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted model to make predictions on the train dataset
train_predictions = classifier.predict(X_train)

print('Model performance on the train set:')
print(confusion_matrix(y_train, train_predictions))
print(classification_report(y_train, train_predictions))
print("Train accuracy:", accuracy_score(y_train, train_predictions))  

Model performance on the train set:
[[   0    2    6    4    3    0    0]
 [   0   75   27   28    8    0    0]
 [   0   31  972  134   26    3    0]
 [   0   23  149 1509   67   18    0]
 [   0    2   46  119  503   18    0]
 [   0    5   18   29   26   62    0]
 [   0    0    0    2    3    0    0]]
              precision    recall  f1-score   support

           3       0.00      0.00      0.00        15
           4       0.54      0.54      0.54       138
           5       0.80      0.83      0.82      1166
           6       0.83      0.85      0.84      1766
           7       0.79      0.73      0.76       688
           8       0.61      0.44      0.51       140
           9       0.00      0.00      0.00         5

    accuracy                           0.80      3918
   macro avg       0.51      0.49      0.50      3918
weighted avg       0.79      0.80      0.79      3918

Train accuracy: 0.796579887697805


In [32]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Get test data to test the classifier
X_test = test_data[model_features]
y_test = test_data[model_target]

# Use the fitted model to make predictions on the test dataset
# Test data going through the Pipeline it's first imputed (with means from the train), scaled (with the min/max from the train data), and finally used to make predictions
test_predictions = classifier.predict(X_test)

print('Model performance on the test set:')
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))
print("Test accuracy:", accuracy_score(y_test, test_predictions))

Model performance on the test set:
[[  0   0   2   3   0   0]
 [  0   8   9   8   0   0]
 [  0  12 187  76  12   4]
 [  0  10 101 261  50  10]
 [  0   1  16  67  90  18]
 [  0   0   4  10  10  11]]
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         5
           4       0.26      0.32      0.29        25
           5       0.59      0.64      0.61       291
           6       0.61      0.60      0.61       432
           7       0.56      0.47      0.51       192
           8       0.26      0.31      0.28        35

    accuracy                           0.57       980
   macro avg       0.38      0.39      0.38       980
weighted avg       0.57      0.57      0.57       980

Test accuracy: 0.5683673469387756
